# Ulysses Narrative GMM Turning Points (minimal)

このノートブックは `scripts/run_report.py` と同等の処理を、Colabで実行する最小版である。


In [23]:
%cd /content
!rm -rf Mokafe-ulysses-narrative-gmm
!git clone https://github.com/Mokafe/Mokafe-ulysses-narrative-gmm.git
%cd /content/Mokafe-ulysses-narrative-gmm
!pip -q install -r requirements.txt

import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / "src"))

import narrative_gmm
print("OK import narrative_gmm:", narrative_gmm.__file__)


/content
Cloning into 'Mokafe-ulysses-narrative-gmm'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 719.10 KiB | 2.88 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/Mokafe-ulysses-narrative-gmm
OK import narrative_gmm: /content/Mokafe-ulysses-narrative-gmm/src/narrative_gmm/__init__.py


In [24]:
from narrative_gmm.report import build_report, ReportConfig
print("OK: imported narrative_gmm.report")


OK: imported narrative_gmm.report


生成された `../outputs/preds_all.csv` には **entropy** と **margin** が含まれる（後段の集計で必須）。
